In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


documents=pd.read_csv('JobAccountant_Cleaned.csv')

documents['Tokenized_Description_Test'] = documents['Tokenized_Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
documents_df=pd.DataFrame(documents,columns=['Tokenized_Description_Test'])


[nltk_data] Downloading package stopwords to C:\Users\MASTER
[nltk_data]     ILYAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
documents

,id,Job Title,Tokenized_Description,Tokenized_Description_Test
0,1,Accountant,benefits working hours monday to friday am-pm ...,benefits working hours monday friday am-pm wor...
1,1,Accountant,if you do not possess the above experience you...,possess experience application still considere...
2,1,Accountant,please submit your updated resume in ms format...,please submit updated resume ms format using a...
3,1,Accountant,alternatively you can send your resumes to mci...,alternatively send resumes mci@mci.com.sg.
4,1,Accountant,our dedicated and approachable mci consultants...,dedicated approachable mci consultants get bac...
...,...,...,...,...
1271,165,Accountant,handling full functions of finance and account...,handling full functions finance accounts leadi...
1272,165,Accountant,we regret that only shortlisted candidates wil...,regret shortlisted candidates notified.
1273,165,Accountant,job code sans email address cap.ta@capitasinga...,job code sans email address cap.ta@capitasinga...
1274,165,Accountant,c,c


In [4]:
documents_df

,Tokenized_Description_Test
0,benefits working hours monday friday am-pm wor...
1,possess experience application still considere...
2,please submit updated resume ms format using a...
3,alternatively send resumes mci@mci.com.sg.
4,dedicated approachable mci consultants get bac...
...,...
1271,handling full functions finance accounts leadi...
1272,regret shortlisted candidates notified.
1273,job code sans email address cap.ta@capitasinga...
1274,c


In [5]:
stop_words_l=stopwords.words('english')
documents_df['Tokenized_Description_Test_Cleaned']=documents['Tokenized_Description_Test'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split()))

In [6]:
documents_df=pd.DataFrame(documents_df,columns=['Tokenized_Description_Test_Cleaned'])

In [7]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [8]:
documents_df

,Tokenized_Description_Test_Cleaned
0,benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply
1,possess experience application still considered individual merits may contacted opportunities
2,please submit updated resume ms format using apply button
3,alternatively send resumes mci mci com sg
4,dedicated approachable mci consultants get back soon confidential discussion
...,...
1271,handling full functions finance accounts leading singapore regional offices preparation financial management reports preparation short long term liquidity planning performing month end closing requirements candidate must possess least diploma accounts finance least years working experience accounts finance field required regional experience must interested candidates wish apply advertised position please email us updated copy resume attnsans contact office confidential discussion
1272,regret shortlisted candidates notified
1273,job code sans email address cap ta capitasingapore com ea license no
1274,c


### Corpus Dataframe

In [9]:
documents_df['Tokenized_Description_Test_Cleaned']

0       benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply                                                                                                                                                           
1       possess experience application still considered individual merits may contacted opportunities                                                                                                                                                                                                                                                                                                                  

In [10]:
tokensized_jd = documents_df.loc[0, 'Tokenized_Description_Test_Cleaned']
print(tokensized_jd)

benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply 


#### Tf-idf vectors

Compute tf-idf weights and the cosine similarity score between two vectors.

Term frequency-inverse document frequency
Proportional to term frequency
Inverse function of the number of documents in which it occurs

In [11]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(documents_df.Tokenized_Description_Test_Cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.Tokenized_Description_Test_Cleaned)

In [12]:
parse_tweets=[]

for i in range(len(documents_df['Tokenized_Description_Test_Cleaned'])) :

        patt=r'https?://\S+'
        rt=re.compile(patt)
        parse_tweets.append(rt.sub('',documents_df['Tokenized_Description_Test_Cleaned'][i]))

parse_tweets[:2]    

['benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply ',
 'possess experience application still considered individual merits may contacted opportunities ']

In [13]:
# Count Word Freq. 
from sklearn.feature_extraction.text import CountVectorizer

# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(parse_tweets)

# OPTIONAL: Convert Sparse Matrix to DF
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names()) 

df.head(9)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abdullah,ability,able,abnormal,abreast,absence,absorption,acca,accepted,access,accomplishing,accordance,according,accordingly,account,accountabilities,accountacy,accountan,accountancy,accountant,accountantpa,accountants,accounted,accounting,accounts,accpac,accrual,accruals,accuracy,accurate,accurately,achieve,achievegroup,achievements,acknowledge,acquired,acquisition,across,act,action,...,withdraw,withholding,within,witholding,wk,wong,word,words,work,worked,worker,workflow,working,workings,works,workshops,wotk,would,write,writing,written,wu,www,xbrl,xi,xin,xiong,yardi,year,yearend,yearly,years,yearsrelevantexperienceinfull,yee,yi,yin,ying,you,yu,zheng
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [14]:
# Actually doing the cosine similarity comparisons now:

from sklearn.metrics.pairwise import cosine_similarity

dj=pd.DataFrame(cosine_similarity(df, dense_output=True))

dj.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275
0,1.000000,0.079057,0.041667,0.000000,0.000000,0.049029,0.0,0.00000,0.247829,0.0,0.0,0.236111,0.0,0.000000,0.0,0.0,0.27735,0.036084,0.0,0.044194,0.072169,0.051031,0.139754,0.073721,0.0,0.0,0.088388,0.088388,0.029463,0.0,0.0,0.062815,0.127632,0.070767,0.049670,0.000000,0.000000,0.0,0.108253,0.075378,...,0.042875,0.0,0.000000,0.024056,0.0,0.113408,0.120281,0.252591,0.044194,0.125000,0.030317,0.0,0.000000,0.0,0.0,0.026064,0.068465,0.068465,0.029463,0.03125,0.051031,0.047246,0.0,0.111803,0.044194,0.041667,0.176777,0.0,0.0,0.100223,0.0,0.0,0.199007,0.027277,0.0625,0.236111,0.0,0.000000,0.0,0.106000
1,0.079057,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.075235,0.0,0.0,0.105409,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.070711,0.046625,0.0,0.0,0.000000,0.000000,0.074536,0.0,0.0,0.000000,0.000000,0.000000,0.041885,0.000000,0.141421,0.0,0.000000,0.000000,...,0.108465,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.158114,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.111803,0.105409,0.111803,0.0,0.0,0.084515,0.0,0.0,0.031466,0.000000,0.0000,0.105409,0.0,0.000000,0.0,0.033520
2,0.041667,0.000000,1.000000,0.000000,0.000000,0.130744,0.0,0.00000,0.132175,0.0,0.0,0.148148,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.294884,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.201008,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.323381,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.148148,0.0,0.000000,0.0,0.035333
3,0.000000,0.000000,0.000000,1.000000,0.222222,0.000000,0.0,0.18490,0.079305,0.0,0.0,0.000000,0.0,0.096225,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.049147,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.100504,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.096225,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.096225,0.0,0.000000
4,0.000000,0.000000,0.000000,0.222222,1.000000,0.000000,0.0,0.09245,0.000000,0.0,0.0,0.074074,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.161690,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.074074,0.0,0.000000,0.0,0.000000


In [15]:
t=[]

# Part 01:
for j,k in enumerate(dj.values):
    for n in range(len(k)):
        t.append([j,n,k[n]])

# Part 02:
qq=[]
for i in range(len(t)):
    if t[i][0]==t[i][1]:
        qq.append([t[i][0],t[i][1],0])
    else:
        qq.append(t[i])
qq[:5]

[[0, 0, 0],
 [0, 1, 0.07905694150420949],
 [0, 2, 0.041666666666666664],
 [0, 3, 0.0],
 [0, 4, 0.0]]

In [16]:
from collections import defaultdict
u=defaultdict(list)

# Part 01:

for i in range(len(qq)):
    u[qq[i][0]].append(qq[i][2])
    
updated_df=pd.DataFrame(u)

# updated_df.max(axis=1)
# max(updated_df[0])
# np.argmax(updated_df[3])
# updated_df[3]

# Part 02:

position_maxVal=[]
for i in range(len(updated_df)):
    position_maxVal.append(np.argmax(updated_df[i]))

In [17]:
sent_comp=[]


for j in position_maxVal: # list of highest similarity index positions
# this creates in order our tweets w/ highest similiarity by row    
            sent_comp.append(parse_tweets[j])
sent_comp

# tweets based on highest similarity value per row as DF
similar_tweets_=pd.DataFrame(sent_comp,columns=['Similar Tweet'])

# similiarity values rounded 4 decimal places finding max value per row
similarity_value_=pd.DataFrame(round(updated_df.max(axis=1),4),
                               columns=['Similarity Value'])

In [18]:
# tweets w/o html in them
p_twt=pd.DataFrame(parse_tweets,columns=['Parsed Tweets'])

# put everything together
cos_sim_df=pd.concat([p_twt,similar_tweets_,similarity_value_],axis=1)


#cos_sim_df.sort_values(by=['Similarity Value'],ascending=False)

cos_sim_df.head(100)



,Parsed Tweets,Similar Tweet,Similarity Value
0,benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply,benefits summary permanent position salary working hours monday friday am pm location bedok responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast liaise shared service transactional entries quarterly gst submission review implement oversee accounting controls policies procedures oversee sox internal control processing withholding tax accounting related duties assigned requirements years relevant experience accounting experience sap advantageous applicants possess relevant experience responsibilities welcome apply,0.6976
1,possess experience application still considered individual merits may contacted opportunities,possess experience application still considered individual merits may contacted opportunities,1.0000
2,please submit updated resume ms format using apply button,please submit updated resume ms format using apply button,1.0000
3,alternatively send resumes mci mci com sg,alternatively send resumes mci mci com sg,1.0000
4,dedicated approachable mci consultants get back soon confidential discussion,dedicated approachable mci consultants get back soon confidential discussion,1.0000
...,...,...,...
95,requirements possess least degree accounting acca preferably least years working experience related field experience full set accounting proficient quickbook accounting software experience familiar auditing process willing take initiatives able work independently take additional responsibilities good analytical skills well organize able instruction clearly willing travel overseas required kindly apply complete resume primary job responsibiliies monthly payroll cpf submission less headcounts full set accounts entities setup updating transaction using quickbook issuing cheque internal external payments setup proper accounting filing system directors expenses claims able handle annual auditing assist executive director daily administration basic administrative work office inventory control petty cash,requirements diploma accounting solid finance accounting knowledge min two years accounting experience including full cycle work,0.3784
96,accountants job scope handle aspects financial accounting matters,financial statutory reporting ensure accurate timely delivery monthly quarterly annual financial reports including analysis commentaries variances ensure financial statutory reports compliance sfrsi preparation accounting position papers financial instruments including derivatives non financial investments implementation new accounting standards matters assist internal external audit matters company audit periods bachelor degree accountancy acca minimum years experience audit and or accounting large singapore organisation experience sap equivalent erp system excellent excel skills strong working knowledge ifrs accounting reporting,0.3465
97,execute day to day operation accounting,prepare assist day to day operation accounting prepare gst returns prepare prepaid accrual schedules bank reconciliation interact internal external auditors completing audits,0.4518
98,ensure proper month end closing accounts timely basis,job description incumbent work team perform daily operations ensure timely closing accounts month end,0.5669


In [19]:
print(cos_sim_df['Parsed Tweets'][1])
print('-----------------')
print(cos_sim_df['Similar Tweet'][2])
print('-----------------')
print('Similarity Value:',cos_sim_df['Similarity Value'][2])    

possess experience application still considered individual merits may contacted opportunities 
-----------------
please submit updated resume ms format using apply button 
-----------------
Similarity Value: 1.0


In [21]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(documents_df.Tokenized_Description_Test_Cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.Tokenized_Description_Test_Cleaned)

In [22]:
documents_df.Tokenized_Description_Test_Cleaned

0       benefits working hours monday friday am pm working location nearby bedok mrt aws times vb basic salary responsibilities handle full set accounts ap ar gl monthly closing review reconcile intercompany transactions discrepancies assist preparation management reports cost ap control b s analysis preparation monthly forecast additional information sap knowledge applicants possess relevant experience responsibilities welcome apply                                                                                                                                                           
1       possess experience application still considered individual merits may contacted opportunities                                                                                                                                                                                                                                                                                                                  

In [23]:
tfidf_vectors.shape

(1276, 64)

In [24]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.         0.         0.18724798 0.         0.19817938 0.18891775
 0.         0.         0.19182558 0.         0.         0.21860695
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.14621936 0.         0.
 0.         0.         0.         0.         0.203139   0.
 0.         0.18563014 0.         0.33715681 0.         0.
 0.         0.         0.         0.19123087 0.41399064 0.
 0.         0.         0.19554406 0.         0.18948646 0.
 0.         0.37020368 0.         0.         0.17596605 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.38012291 0.        ]


##### Every document has been converted into a 64 dimensional vector. As we set the max_features=64